In [1]:
include("foldexity.jl")

In [2]:
pdb = readpdb("testpdb/small.pdb")
writepdb(pdb, "testpdb/small_backbone.pdb")

xyzcoords = pdb2matrix(pdb)
coordmatrix = matrix2fragments(xyzcoords, 12)

pdbmatrix = pdb2pdbmatrix(pdb)
pdbfragments = matrix2fragments(pdbmatrix)

println()


In [ ]:
function missing_residues(pdb)
    resid = unique(pdb.resid)
    return !all((diff(resid) .== 1))  
end



false

Int32[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [3]:
pfrag1 = coordmatrix[10]
pfrag2 = coordmatrix[31]

align_rmsd(pfrag1, pfrag2)

19.746475496483697

In [4]:
aver_rmsd, nclusts, fxity, nres, matrix = fxity_kabsh(coordmatrix)

(12.604134669018569, 37, 0.9487179487179487, 39, [0.0 15.259918127243338 … 15.673347910584875 6.248241906285712; 15.259918127243338 0.0 … 17.808255124041203 14.988237305484544; … ; 15.673347910584875 17.808255124041203 … 0.0 14.628995374500345; 6.248241906285712 14.988237305484544 … 14.628995374500345 0.0])

In [5]:
fxity, nfrags = fxpdb("testpdb/small.pdb") 
data = fxdir("testpdb/", "tmp", 4, true)  
rm("tmp")

Starting foldexity ...
5 files are collected 
Starting foldexity ...
1	testpdb/3EFM.pdb	0.5202108963093146	296	4.470485886057379	569
2	testpdb/AF-Q8VCK6-F1-model_v4.pdb	0.26605504587155965	87	2.6608642385790833	327
3	testpdb/polya.pdb	0.1724137931034483	10	0.364438980618678	58
4	testpdb/small.pdb	0.2978723404255319	14	2.208886731628906	47
5	testpdb/small_backbone.pdb	0.2978723404255319	14	2.208886731628906	47
